In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
                               '12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


**41.** Delete the From_To column from `df` and attach the temporary DataFrame from the previous questions.

In [7]:
temp = df.From_To.str.split('_', expand=True)
temp.columns = ['From', 'To']
temp['From'] = temp['From'].str.capitalize()
temp['To'] = temp['To'].str.capitalize()

AttributeError: 'DataFrame' object has no attribute 'From_To'

In [ ]:
df = df.drop('From_To', axis=1)
df = df.join(temp)

**42.** In the Airline column, you can see some extra puctuation and symbols have appeared around the airline names. Pull out just the airline name. E.g. '(British Airways. )' should become 'British Airways'.

In [8]:
df['Airline'] = df['Airline'].str.extract('([a-zA-Z\s]+)', expand=False).str.strip()
df

,FlightNumber,RecentDelays,Airline
0,10045.0,"[23, 47]",KLM
1,NaN,[],Air France
2,10065.0,"[24, 43, 87]",British Airways
3,NaN,[13],Air France
4,10085.0,"[67, 32]",Swiss Air


**43.** In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.

Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [9]:
delays = df['RecentDelays'].apply(pd.Series)

In [10]:
delays.columns = ['delay_{}'.format(n) for n in range(1, len(delays.columns)+1)]

In [11]:
df = df.drop('RecentDelays', axis=1).join(delays)
df

,FlightNumber,Airline,delay_1,delay_2,delay_3
0,10045.0,KLM,23.0,47.0,NaN
1,NaN,Air France,NaN,NaN,NaN
2,10065.0,British Airways,24.0,43.0,87.0
3,NaN,Air France,13.0,NaN,NaN
4,10085.0,Swiss Air,67.0,32.0,NaN


**44.** Given the lists letters = ['A', 'B', 'C'] and numbers = list(range(10)), construct a MultiIndex object from the product of the two lists. Use it to index a Series of random numbers. Call this Series s.

In [12]:
letters = ['A', 'B', 'C']
numbers = list(range(10))

In [13]:
MultiIndex = pd.MultiIndex.from_product([letters, numbers])

In [15]:
series = pd.Series(np.random.rand(30), index=MultiIndex)
series

A  0    0.312412
   1    0.988079
   2    0.401333
   3    0.641062
   4    0.294149
   5    0.524961
   6    0.611959
   7    0.128071
   8    0.156497
   9    0.576138
B  0    0.916686
   1    0.939019
   2    0.393478
   3    0.838244
   4    0.802007
   5    0.106385
   6    0.616234
   7    0.010251
   8    0.212396
   9    0.756822
C  0    0.522265
   1    0.129311
   2    0.202777
   3    0.704402
   4    0.236451
   5    0.524738
   6    0.293154
   7    0.575519
   8    0.899677
   9    0.111854
dtype: float64

**45.** Check the index of s is lexicographically sorted (this is a necessary proprty for indexing to work correctly with a MultiIndex).

In [17]:
series.index.is_lexsorted()


True

**46.** Select the labels 1, 3 and 6 from the second level of the MultiIndexed Series

In [18]:
series.loc[:, [1, 3, 6]]

A  1    0.988079
   3    0.641062
   6    0.611959
B  1    0.939019
   3    0.838244
   6    0.616234
C  1    0.129311
   3    0.704402
   6    0.293154
dtype: float64

**47.** Slice the Series s; slice up to label 'B' for the first level and from label 5 onwards for the second level.

In [19]:
series.loc[pd.IndexSlice[:'B', 5:]]

A  5    0.524961
   6    0.611959
   7    0.128071
   8    0.156497
   9    0.576138
B  5    0.106385
   6    0.616234
   7    0.010251
   8    0.212396
   9    0.756822
dtype: float64

**48.** Sum the values in s for each label in the first level (you should have Series giving you a total for labels A, B and C).

In [20]:
series.sum(level=0)

A    4.634661
B    5.591522
C    4.200148
dtype: float64

**49.** Suppose that sum() (and other methods) did not accept a level keyword argument. How else could you perform the equivalent of s.sum(level=1)?

In [21]:
series.unstack().sum(axis=0)

0    1.751364
1    2.056409
2    0.997588
3    2.183708
4    1.332607
5    1.156083
6    1.521347
7    0.713841
8    1.268570
9    1.444813
dtype: float64

**50.** Exchange the levels of the MultiIndex so we have an index of the form (letters, numbers). Is this new Series properly lexsorted? If not, sort it.

In [22]:
new_series = series.swaplevel(0, 1)

In [23]:
new_series.index.is_lexsorted()

False

In [24]:
new_series = new_series.sort_index()

In [26]:
new_series

0  A    0.312412
   B    0.916686
   C    0.522265
1  A    0.988079
   B    0.939019
   C    0.129311
2  A    0.401333
   B    0.393478
   C    0.202777
3  A    0.641062
   B    0.838244
   C    0.704402
4  A    0.294149
   B    0.802007
   C    0.236451
5  A    0.524961
   B    0.106385
   C    0.524738
6  A    0.611959
   B    0.616234
   C    0.293154
7  A    0.128071
   B    0.010251
   C    0.575519
8  A    0.156497
   B    0.212396
   C    0.899677
9  A    0.576138
   B    0.756822
   C    0.111854
dtype: float64